In [ ]:
import numpy as np
from time import time
from loguru import logger
import sys

In [2]:
raw_data = open('files/input_day7.txt').read()

In [3]:
example = '''.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............'''

In [4]:
class TachyonManifold:
    def __init__(self, raw_data, is_example = False):
        self.is_example = is_example
        logger.remove()
        if self.is_example:
            logger.add(sys.stderr, level="DEBUG")
        else:
            logger.add(sys.stderr, level="INFO")
        self.diagram = self.preprocess(raw_data)
        self.start = self.find_start()
        self.n_splitted = 0

    def preprocess(self, data):    
        return data.strip().split('\n')

    def find_start(self):
        for i, row in enumerate(self.diagram):
            if 'S' in row:
                return i, row.index('S')
    
    def find_beams_in_row(self, row):
        return [i for i, c in enumerate(row) if c == '|']

    def set_beam(self, r, c):
        l = self.diagram[r]
        self.diagram[r] = l[:c] + '|' + l[c+1:]

    def forward_beam(self, r, c):
        if r == len(self.diagram) - 1:
            return
        is_splitter = self.diagram[r+1][c] == '^'
        if is_splitter:
            if c-1 >= 0:
                self.set_beam(r+1, c-1)
            if c+1 < len(self.diagram):
                self.set_beam(r+1, c+1)
            self.n_splitted += 1
            logger.debug(f'Splitted now {self.n_splitted} times')
        else:
            self.set_beam(r+1, c)

    def analyze(self):
        t_start = time()

        # Set starting beam
        self.set_beam(self.start[0]+1, self.start[1])
        # Loop through rows
        for r, row in enumerate(self.diagram):
            beams_in_row = self.find_beams_in_row(row)
            for c in beams_in_row:
                self.forward_beam(r, c)
            logger.debug('\n'+'\n'.join(self.diagram))

        t_end = time()
        logger.info(f'Part 1 took: {(t_end-t_start)*1000:.2f}ms')
        logger.info(f'Result is n times splitted {self.n_splitted}')

ex_man = TachyonManifold(example, is_example = True)
ex_man.analyze()

2025-12-08 17:51:46.692 | DEBUG    | __main__:analyze:52 - 
.......S.......
.......|.......
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
2025-12-08 17:51:46.692 | DEBUG    | __main__:forward_beam:38 - Splitted now 1 times
2025-12-08 17:51:46.692 | DEBUG    | __main__:analyze:52 - 
.......S.......
.......|.......
......|^|......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
2025-12-08 17:51:46.693 | DEBUG    | __main__:analyze:52 - 
.......S.......
.......|.......
......|^|......
......|.|......
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............

In [5]:
man = TachyonManifold(raw_data, is_example = False)
man.analyze()

2025-12-08 17:51:46.709 | INFO     | __main__:analyze:55 - Part 1 took: 3.03ms
2025-12-08 17:51:46.709 | INFO     | __main__:analyze:56 - Result is n times splitted 1560


# Part 2

In [65]:
class QuantumTachyonManifold:
    def __init__(self, raw_data, is_example = False):
        self.is_example = is_example
        logger.remove()
        if self.is_example:
            logger.add(sys.stderr, level="DEBUG")
        else:
            logger.add(sys.stderr, level="INFO")
        self.diagram = self.preprocess(raw_data)
        self.start = self.find_start()
        self.n_splitted = 0

    def preprocess(self, data):    
        return [list(row) for row in data.strip().split('\n')]

    def find_start(self):
        for i, row in enumerate(self.diagram):
            if 'S' in row:
                return i, row.index('S')
    
    def find_beams_in_row(self, row):
        return [i for i, c in enumerate(row) if c != '^' and c != '.' and c != 'S']

    def set_beam(self, r, c, mult:int = 1):
        l = self.diagram[r]
        if l[c] == '.' or l[c] == 'S':
            n_beans = mult
        elif l[c] == '^':
            logger.error('WTF')
        else:
            n_beans = int(l[c]) + mult
        new_n_beans_str = str(n_beans)
        self.diagram[r][c] = new_n_beans_str

    def forward_beam(self, r, c):
        if r == len(self.diagram) - 1:
            return
        is_splitter = self.diagram[r+1][c] == '^'
        beam_mult = int(self.diagram[r][c])
        if is_splitter:
            if c-1 >= 0:
                self.set_beam(r+1, c-1, beam_mult)
            if c+1 < len(self.diagram):
                self.set_beam(r+1, c+1, beam_mult)
            self.n_splitted += 1
            logger.debug(f'Splitted now {self.n_splitted} times')
        else:
            self.set_beam(r+1, c, beam_mult)


    def analyze(self):
        t_start = time()

        # Set starting beam
        self.set_beam(self.start[0]+1, self.start[1], 1)
        self.show_diagram()
        # Loop through rows
        for r, row in enumerate(self.diagram):
            if r == 0:
                continue
            logger.debug(f'Analyzing row {r}: {row}')
            pos_of_beams_in_row = self.find_beams_in_row(row)
            timelines_list = [int(row[pos]) for pos in pos_of_beams_in_row]
            timelines = sum(timelines_list)
            logger.debug(f'Row has beams {timelines_list} so timelines up to now are {timelines}')
            for c in pos_of_beams_in_row:
                logger.debug(f'Beam in pos {r, c} is {self.diagram[r][c]}')
                self.forward_beam(r, c)
            self.show_diagram()

        t_end = time()
        logger.info(f'Part 2 took: {(t_end-t_start)*1000:.2f}ms')
        logger.info(f'Result is n timelines = {timelines}')

    def show_diagram(self):
        logger.debug('\n'+'\n'.join([''.join(row) for row in self.diagram]))

ex_man = QuantumTachyonManifold(example, is_example = True)
ex_man.show_diagram()

2025-12-08 18:30:22.403 | DEBUG    | __main__:show_diagram:76 - 
.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............


In [66]:
ex_man.analyze()

2025-12-08 18:30:23.765 | DEBUG    | __main__:show_diagram:76 - 
.......S.......
.......1.......
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
2025-12-08 18:30:23.766 | DEBUG    | __main__:analyze:61 - Analyzing row 1: ['.', '.', '.', '.', '.', '.', '.', '1', '.', '.', '.', '.', '.', '.', '.']
2025-12-08 18:30:23.766 | DEBUG    | __main__:analyze:65 - Row has beams [1] so timelines up to now are 1
2025-12-08 18:30:23.767 | DEBUG    | __main__:analyze:67 - Beam in pos (1, 7) is 1
2025-12-08 18:30:23.767 | DEBUG    | __main__:forward_beam:46 - Splitted now 1 times
2025-12-08 18:30:23.767 | DEBUG    | __main__:show_diagram:76 - 
.......S.......
.......1.......
......1^1......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
............

In [67]:
Qman = QuantumTachyonManifold(raw_data, is_example = False)
Qman.analyze()

2025-12-08 18:30:48.667 | INFO     | __main__:analyze:72 - Part 2 took: 20.18ms
2025-12-08 18:30:48.668 | INFO     | __main__:analyze:73 - Result is n timelines = 25592971184998
